In [1]:
import spacy
import pandas as pd

In [2]:
nlp = spacy.load("en_core_web_md")

# Amostrando dados para análise

In [3]:
data_df = pd.read_csv("../data/tech_test.tsv", sep="\t").sample(100)

data_df = data_df[["review_body", "review_user_rating"]]

In [4]:
sample_txts = data_df["review_body"].values
# sample_txts = data_df[data_df["review_user_rating"] == 5]["review_body"].values

# Identificação de frases que se referem ao produto
Identificação de frases em que a palavra "refrigerator" aparece como sujeito.

In [5]:
reference = nlp("refrigerator")

In [6]:
i = 0
sentences = []
for txt in sample_txts:
    doc = nlp(txt)
    for sent in doc.sents:
        for token in sent:
            txt = token.text
            if token.dep_ in ("nsubj", "nsubjpass") and (txt.lower() == "it" or token.similarity(reference) > 0.85):
                print(sent)
                sentences.append(sent)
print(len(sentences))

If it continues I will call for service.
We bought this as an emergency replacement, it is larger than our old one, has a lot more room in both the fridge and the freezer.
The sshelves are easy to clean and the LED light is so much brighter, makes it easy to find things in both the fridge and the freezer.
We have a nice size kitchen but our fridge and island felt too close with our old fridge.
It's perfect!
The only reason I didn't give it 5 stars is because it gets a little tight on the inside during the holidays and birthdays.
From the outside the refrigerator is quite appealing however it is too noisy.  
The drip tray for water does not have any holes to catch excess water so it rests on top if it drips.
The drip tray for water does not have any holes to catch excess water so it rests on top if it drips.
It has alot of room and shelf space.
It simply gave up it's freon one day.
It simply gave up it's freon one day.
I had this refrigerator installed about a month ago.
I've never had 

# Extrair adjetivos e modificadores de objetos das frases obtidas

In [7]:
from spacy.matcher import Matcher

In [8]:
matcher = Matcher(nlp.vocab)

In [9]:
# pattern1 = [
#     {'DEP': 'ROOT'},
#     {'POS': 'ADV', 'OP':"?"},
#     {'POS': 'DET', 'OP':"?"},
#     {'DEP': "npadvmod", "OP": "?"},
#     {'DEP':'amod', 'OP':"?"},
#     {'POS':'NOUN'}
# ]

# pattern2 = [
#     {'DEP': 'ROOT'},
#     {'POS': 'ADV', 'OP':"?"},
#     {'POS': 'DET', 'OP':"?"},
#     {'DEP': "npadvmod", "OP": "?"},
#     {'DEP':'amod', 'OP':"?"},
#     {'POS': "ADJ"}
# ]


# pattern1 = [
#     {'POS': 'ADV', 'OP':"?"},
#     {'POS': 'DET', 'OP':"?"},
#     {'DEP': "npadvmod", "OP": "?"},
#     {'POS':'NOUN'}
# ]

pattern2 = [
    # {'POS': 'DET', 'OP':"?"},
    # {'DEP': "npadvmod", "OP": "?"},
    # {'DEP':'amod', 'OP':"?"},
    {'POS': "ADJ"},
    {'POS':'NOUN', "OP": "?"}
]

# matcher.add("description_matcher", None, pattern1) 
matcher.add("adj_matcher", None, pattern2) 

In [10]:
for sent in sentences:
    print(sent)
    matches = matcher(sent)
    # print(matches)
    if len(matches) > 0:
        print("- Matches:")
        for match in matches:
            print("\t ",sent[match[1]:match[2]])
    # print(matcher(sent))
    print()

If it continues I will call for service.

We bought this as an emergency replacement, it is larger than our old one, has a lot more room in both the fridge and the freezer.
- Matches:
	  larger
	  old
	  old one
	  more
	  more room

The sshelves are easy to clean and the LED light is so much brighter, makes it easy to find things in both the fridge and the freezer.
- Matches:
	  easy
	  brighter
	  easy

We have a nice size kitchen but our fridge and island felt too close with our old fridge.
- Matches:
	  nice
	  nice size
	  close
	  old
	  old fridge

It's perfect!
- Matches:
	  perfect

The only reason I didn't give it 5 stars is because it gets a little tight on the inside during the holidays and birthdays.
- Matches:
	  only
	  only reason
	  little
	  tight

From the outside the refrigerator is quite appealing however it is too noisy.  
- Matches:
	  appealing
	  noisy

The drip tray for water does not have any holes to catch excess water so it rests on top if it drips.
- Match

In [11]:
doc = nlp("strange noises")
for tok in doc:
    print("{}, {}, {}".format(tok, tok.dep_, tok.pos_))

strange, amod, ADJ
noises, ROOT, NOUN


In [12]:
doc[0].is_stop

False

In [13]:
from textacy.extract import subject_verb_object_triples, entities, ngrams, noun_chunks, semistructured_statements

In [14]:
for sent in sentences:
    print(list(subject_verb_object_triples(sent)))
    # print(list(entities(sent)))

[]
[(We, bought, this)]
[]
[]
[]
[(I, didn't give, it), (I, didn't give, stars)]
[]
[]
[]
[]
[(It, gave, freon)]
[(It, gave, freon)]
[]
[(I, 've never had, water dispenser)]
[]
[]
[]
[(that, upset, me), (that, upset, most), (it, kept, coming)]
[]
[]
[]
[]
[]
[(I, like, fact)]
[]
[(It, does make, noise)]
[(It, does make, noise)]
[]
[]
[]
[]
[]
[]
[(it, is made, to hold)]
[(it, is made, to hold)]
[(It, fits, bottles)]
[(It, does not accommodate, ounce cottage cheeses), (It, does not accommodate, cream)]
[]
[]
[]
[(refrigerator, stick, foot), (refrigerator, stick, sand)]
[]
[(you, opened, door), (which, didn't take, space)]
[]
[]
[(ice, frees, space), (appliance, runs, lot)]
[]
[(it, leaked, water)]
[(ice dispenser, jams, lot)]
[]
[]
[]
[(they, try, to fix)]
[(I, used, fridge)]
[]
[]
[(one, keeps, things), (one, keeps, frozen)]
[]
[]
[]
[]
[]
[]
[(I, like, the), (it, saves, lot)]
[]
[(I, used, ice)]
[]
[]
[(we, love, shelving), (we, love, drawers)]
[]
[(I, needed, to find), (unit, fit, sp

In [15]:
raw_text = "\n".join(sample_txts)

In [16]:
list(ngrams(nlp(raw_text), n=3, include_pos=["ADJ","AUX", "NOUN", "VERB", "DET", "CONJ"] ,min_freq=1))

[LOVE LOVE LOVE,
 LOVE our new,
 buy more groceries,
 review was collected,
 ice crystals forming,
 love the look,
 French door unit,
 FIXING the issue,
 Needed a fridge,
 counter depth refrigerator,
 dual ice makers,
 review was collected,
 lot more room,
 sshelves are easy,
 review was collected,
 nice size kitchen,
 chose counter depth,
 little more room,
 gets a little,
 good thing tetras,
 review was collected,
 interior is bright,
 outside the refrigerator,
 catch excess water,
 review was collected,
 live the ice,
 review was collected,
 refrigerator a month,
 review was collected,
 review was collected,
 lock is easy,
 open door alarm,
 review was collected,
 like the overall,
 review was collected,
 replace my ancient,
 lighting is excellent,
 previous fridge lasted,
 review was collected,
 love the new,
 new auto fill,
 auto fill feature,
 bought my Frigidaire,
 Frigidaire a couple,
 review was collected,
 like the room,
 Ice machine runs,
 reset new refrigerator,
 bought all

In [17]:
for sent in sentences:
    print(list(noun_chunks(sent)))
    # print(list(entities(sent)))

[it, I, service]
[We, emergency replacement, it, old one, lot more room, both the fridge, freezer]
[sshelves, LED light, it, things, both the fridge, freezer]
[We, nice size kitchen, fridge, island, old fridge]
[It]
[only reason, I, it, 5 stars, it, inside, holidays, birthdays]
[outside, refrigerator, it]
[drip tray, water, holes, excess water, it, top, it]
[drip tray, water, holes, excess water, it, top, it]
[It, alot, room, shelf space]
[It, it]
[It, it]
[I, refrigerator]
[I, ice/water dispenser, it]
[previous fridge, one]
[it, LED lights]
[OPEN FLOOR PLAN HOME REFRIGERATOR, MILLION DOLLAR HOME]
[We, Maytag Ice, Water, door, thing, me, thing, you, ice, it]
[French Door Refrigerator, water, ice, Ice, freezer, top, full cavity, room]
[water, refrigerator, it]
[Overall great fridge, you, enough help, it, flight, stairs]
[it, us]
[great fridge]
[I, fact, it, nice hefty drawers]
[It]
[It, little noise, we, weÃ¢ÂÂll, it, anything]
[It, little noise, we, weÃ¢ÂÂll, it, anything]
[frig, s

In [19]:
for sent in sample_txts:
    print(list(semistructured_statements(nlp(sent), "it")))
    # print(list(entities(sent)))

[]
[]
[]
[]
[(it, is, larger than our old one)]
[]
[]
[]
[(it, is, too noisy)]
[]
[]
[]
[]
[]
[]
[]
[]
[(it, is, nice and bright also with LED lights)]
[]
[]
[]
[]
[(it, is, just as convenient)]
[]
[(it, was, four of us)]
[(it, is, not flimsy also nice hefty drawers)]
[]
[]
[]
[]
[]
[]
[]
[]
[(it, should be, this loud)]
[]
[]
[]
[(it, 's, a cosmetic thing and isn't covered by warranty)]
[]
[(It, is, so tiny), (It, would be, nice if the entire shelf retracted.  )]
[(It, is, loud when the compressor is running and when the ice machine is filling up)]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[(It, 's, quiet, spacious, and very easy to access the food inside, especially the freezer)]
[]
[]
[]
[]
[]
[(it, should be, this loud)]
[]
[]
[]
[(it, 's, so quiet)]
[]
[]
[]
[(It, is, 20.5 cubic feet and we love the clear shelving and drawers)]
[]
[]
[]
[(it, is, attractive, spacious and functional.  )]
[]
[]
[(it, was, well worth the price)]
[]
[]
[]
[]
[(IT, 'S, A GREAT SIZE AND LOTS OF SPACE)]
[]
[]
[]
[]
[]
[